In [1]:
class Operator:
    def __init__(self,unique_id,children,time_required,is_join=False, relative_deadline=False):
#         print ("initialized operator {}".format(unique_id))
        self.children = children
        self.log = []
        self.unique_id = unique_id
        self.time_required = time_required
        self.is_join = is_join
        self.paused_job = None
        self.relative_deadline = relative_deadline
        
class Lattice:
    def __init__(self, operators):
        self.operators = operators
        self.children_dir = {}
        for op in operators:
            self.children_dir[op.unique_id] = op.children 
        # should prolly sort the operators 
    def __repr__(self):
        return "Lattice {}".format(self.children_dir.__repr__())            
    def get_children(self,operator):
        return self.children_dir[operator]
    def add_next(self,event,new_event_queue,curr_time):
        '''
            Adds new events to queue if necessary when an Event completes
            The times are a bit weird cause this effectively gets called at the end of the curr_time slice/beginning of the next slice
        '''
        children = self.get_children(event.operator)
        for c in children:
            c = self.get_op(c)
            event_id = event.unique_id
            if c.is_join:
                if not c.paused_job:
                    c.paused_job = event
                    continue
                else:
                    event_id = min(event.unique_id,c.paused_job.unique_id)
                    c.paused_job = None
            if c.relative_deadline:
                d = c.relative_deadline + curr_time + 1
            else: 
                d = None
            new_event_queue.append(Event(event_id,c.unique_id, curr_time + 1,deadline=d))
    def get_op(self,op):
        '''
            Assumes that the operators are in sorted and consecutive order
        '''
        out = self.operators[op]
        if out.unique_id != op:
            print ("ERROR: lattice not properly ordered [{} returned instead of {}]".format(out.unique_id,op))
        return out



In [2]:
op4 = Operator(4,[],8)
op3 = Operator(3,[4],6,is_join=True)
op2 = Operator(2,[3],4)
op1 = Operator(1,[3],2)
op0 = Operator(0,[1,2],10)

lat = Lattice([op0,op1,op2,op3,op4])

print(lat)
print (lat.get_children(3))

# Lattice {0: [1, 2], 1: [3], 2: [3], 3: [4], 4: []}
# [4]

Lattice {0: [1, 2], 1: [3], 2: [3], 3: [4], 4: []}
[4]


``` 
v1 -> v3 -> v4
^      ^   
|      |
v0 ->  v2
```

In [3]:
            
class Event:
    def __init__(self,  unique_id, operator , arrival_time, deadline=None):
        self.unique_id = unique_id
        self.operator = operator
        self.arrival_time = arrival_time
        self.time_remaining = -1
        self.start_time = None
        self.finish_time = None
        self.deadline = deadline
    def __repr__(self):
        return "<Event {}; Running Op {}; Available at Time {}; Executed {} to {}; Deadline: {}>".format(self.unique_id,self.operator, self.arrival_time, self.start_time, self.finish_time, self.deadline)
    def start(self,lattice,time):
        self.time_remaining = lattice.get_op(self.operator).time_required
        self.start_time = time 
    def step(self):
        if self.time_remaining < 1:
            print("ERROR: already finished event; {} left".format(self.time_remaining))
        self.time_remaining -= 1
        return self.time_remaining
    def finish(self,new_event_queue,lattice,time):
        if self.time_remaining != 0:
            print("ERROR: try to finish but not done; {} left".format(self.time_remaining))
        lattice.add_next(self,new_event_queue,time)
        self.finish_time = time + 1 # cause you technically am using up the current time slice
        if self.deadline and self.deadline < self.finish_time:
            print("WARNING: DEADLINE MISSED [D:{}; F:{}]".format(self.deadline,self.finish_time))
        
        
        
        


In [4]:
event_queue = []
e = Event(1,0,0)
e.start(lat,0)
for i in range(10):
    e.step()
e.finish(event_queue,lat,2)
e,event_queue
# (<Event 1; Running Op 0; Available at Time 0; Executed 0 to 2>,
#  [<Event 1; Running Op 1; Available at Time 2; Executed None to None>,
#   <Event 1; Running Op 2; Available at Time 2; Executed None to None>])


(<Event 1; Running Op 0; Available at Time 0; Executed 0 to 3; Deadline: None>,
 [<Event 1; Running Op 1; Available at Time 3; Executed None to None; Deadline: None>,
  <Event 1; Running Op 2; Available at Time 3; Executed None to None; Deadline: None>])

In [5]:



class Worker: 
    def __init__(self, unique_id, gpus=1):
        self.unique_id = unique_id
        self.history = []
        self.current_event = None
    def __repr__(self):
        return "Worker {}: {}".format(self.unique_id, self.history)
    def do_job(self, task, lattice,time):
        if self.current_event != None:
            print("ERROR: Worker is still busy")
        self.history.append(task)
        self.current_event = task
        task.start(lattice,time)
    def reset(self):
        self.__init__(self.unique_id)
    def get_history(self):
        out = ""
        for e in self.history:
            out += "\n  {}".format(e)
        return "[{}\n]".format(out)
            
        
class WorkerPool:
    def __init__(self, workers):
        self.workers = workers
        self.count = len(workers)
    def __repr__(self):
        output = ""
        for w in self.workers:
            output += "\n{}".format(w)
        return "Worker Pool: " + output
    def add_worker(self,w):
        for v in self.workers:
            if v.unique_id == w.unique_id:
                print ("ERROR: worker not unique [{}]".format(w.unique_id))
                return 
        self.workers.append(w)
    def add_workers(self,ws):
        for w in ws:
            self.add_worker(w)
    def reset(self):
        for w in self.workers:
            w.reset()
    def history(self):
        output = ""
        for w in self.workers:
            output += "\nWorker{}: {}".format(w.unique_id, w.get_history())
        return "Worker Pool History: " + output
        
            
    

In [6]:
w = Worker(1)
w.do_job(Event(7,4,2),lat,2)
w # Worker 1: [<Event 7; Running Op 4; Available at Time 2>]

Worker 1: [<Event 7; Running Op 4; Available at Time 2; Executed 2 to None; Deadline: None>]

In [7]:
wp = WorkerPool([])
v = Worker(1)
wp.add_workers([w,v])
# ERROR: worker not unique [1]

wp
# Worker Pool: 
# Worker 1: [<Event 7; Running Op 4; Available at Time 2>]

ERROR: worker not unique [1]


Worker Pool: 
Worker 1: [<Event 7; Running Op 4; Available at Time 2; Executed 2 to None; Deadline: None>]

In [8]:
def simulate(schedule,task_set,worker_pool,lattice,timeout,v=0):
    if not task_set:
        return
    task_set.sort(key=(lambda e: e.arrival_time))
    event_queue = []
    for time in range(timeout):
        if v:
            print ("step: {}".format(time))
        while len(task_set) != 0 and  task_set[0].arrival_time == time:
            task = task_set.pop(0)
            print ("Activate: {}".format(task))
            event_queue.append(task)
        schedule(time,event_queue,lattice,worker_pool,timeout)
def fifo_schedule(time,event_queue,lattice,worker_pool,timeout):
    new_event_queue = []
#     print (event_queue)
    for worker in worker_pool.workers:
        if worker.current_event == None and event_queue:
            worker.do_job(event_queue.pop(0),lattice,time)
        if worker.current_event: 
            worker.current_event.step()
            if worker.current_event.time_remaining == 0:
                worker.current_event.finish(new_event_queue,lattice,time)
                print("finished event: {}".format(worker.current_event))
                worker.current_event = None
    event_queue.extend(new_event_queue)

def edf_schedule(time,event_queue,lattice,worker_pool,timeout):
    new_event_queue = []
#     print (event_queue)
    for worker in worker_pool.workers:
        if worker.current_event == None and event_queue:
            worker.do_job(event_queue.pop(0),lattice,time)
        if worker.current_event: 
            worker.current_event.step()
            if worker.current_event.time_remaining == 0:
                worker.current_event.finish(new_event_queue,lattice,time)
                print("finished event: {}".format(worker.current_event))
                worker.current_event = None
    event_queue.extend(new_event_queue)
    event_queue.sort(key=lambda x: x.deadline if x.deadline else timeout)
def llf_schedule(time,event_queue,lattice,worker_pool,timeout):
    new_event_queue = []
#     print (event_queue)
    for worker in worker_pool.workers:
        if worker.current_event == None and event_queue:
            worker.do_job(event_queue.pop(0),lattice,time)
        if worker.current_event: 
            worker.current_event.step()
            if worker.current_event.time_remaining == 0:
                worker.current_event.finish(new_event_queue,lattice,time)
                print("finished event: {}".format(worker.current_event))
                worker.current_event = None
    event_queue.extend(new_event_queue)
    event_queue.sort(key=lambda x: x.deadline - x.time_remaining if x.deadline else timeout)


In [9]:
wp1 = WorkerPool([])
wp1.add_worker(v)
wp1

Worker Pool: 
Worker 1: []

In [10]:

simulate(fifo_schedule,[Event(2,0,4),Event(5,0,40)],wp1,lat,100)

Activate: <Event 2; Running Op 0; Available at Time 4; Executed None to None; Deadline: None>
finished event: <Event 2; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: None>
finished event: <Event 2; Running Op 1; Available at Time 14; Executed 14 to 16; Deadline: None>
finished event: <Event 2; Running Op 2; Available at Time 14; Executed 16 to 20; Deadline: None>
finished event: <Event 2; Running Op 3; Available at Time 20; Executed 20 to 26; Deadline: None>
finished event: <Event 2; Running Op 4; Available at Time 26; Executed 26 to 34; Deadline: None>
Activate: <Event 5; Running Op 0; Available at Time 40; Executed None to None; Deadline: None>
finished event: <Event 5; Running Op 0; Available at Time 40; Executed 40 to 50; Deadline: None>
finished event: <Event 5; Running Op 1; Available at Time 50; Executed 50 to 52; Deadline: None>
finished event: <Event 5; Running Op 2; Available at Time 50; Executed 52 to 56; Deadline: None>
finished event: <Event 5; Running Op 

In [11]:
print(wp1.history())

Worker Pool History: 
Worker1: [
  <Event 2; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: None>
  <Event 2; Running Op 1; Available at Time 14; Executed 14 to 16; Deadline: None>
  <Event 2; Running Op 2; Available at Time 14; Executed 16 to 20; Deadline: None>
  <Event 2; Running Op 3; Available at Time 20; Executed 20 to 26; Deadline: None>
  <Event 2; Running Op 4; Available at Time 26; Executed 26 to 34; Deadline: None>
  <Event 5; Running Op 0; Available at Time 40; Executed 40 to 50; Deadline: None>
  <Event 5; Running Op 1; Available at Time 50; Executed 50 to 52; Deadline: None>
  <Event 5; Running Op 2; Available at Time 50; Executed 52 to 56; Deadline: None>
  <Event 5; Running Op 3; Available at Time 56; Executed 56 to 62; Deadline: None>
  <Event 5; Running Op 4; Available at Time 62; Executed 62 to 70; Deadline: None>
]


In [12]:
wp1.reset()
v.history

[]

In [13]:
x = Worker(2)
x.reset()
wp2 = WorkerPool([])
wp2.add_workers([v,x])
wp2

Worker Pool: 
Worker 1: []
Worker 2: []

In [14]:
simulate(fifo_schedule,[Event(2,0,4),Event(5,0,40)],wp2,lat,100)

Activate: <Event 2; Running Op 0; Available at Time 4; Executed None to None; Deadline: None>
finished event: <Event 2; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: None>
finished event: <Event 2; Running Op 1; Available at Time 14; Executed 14 to 16; Deadline: None>
finished event: <Event 2; Running Op 2; Available at Time 14; Executed 14 to 18; Deadline: None>
finished event: <Event 2; Running Op 3; Available at Time 18; Executed 18 to 24; Deadline: None>
finished event: <Event 2; Running Op 4; Available at Time 24; Executed 24 to 32; Deadline: None>
Activate: <Event 5; Running Op 0; Available at Time 40; Executed None to None; Deadline: None>
finished event: <Event 5; Running Op 0; Available at Time 40; Executed 40 to 50; Deadline: None>
finished event: <Event 5; Running Op 1; Available at Time 50; Executed 50 to 52; Deadline: None>
finished event: <Event 5; Running Op 2; Available at Time 50; Executed 50 to 54; Deadline: None>
finished event: <Event 5; Running Op 

In [15]:
print(wp2.history())

Worker Pool History: 
Worker1: [
  <Event 2; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: None>
  <Event 2; Running Op 1; Available at Time 14; Executed 14 to 16; Deadline: None>
  <Event 2; Running Op 3; Available at Time 18; Executed 18 to 24; Deadline: None>
  <Event 2; Running Op 4; Available at Time 24; Executed 24 to 32; Deadline: None>
  <Event 5; Running Op 0; Available at Time 40; Executed 40 to 50; Deadline: None>
  <Event 5; Running Op 1; Available at Time 50; Executed 50 to 52; Deadline: None>
  <Event 5; Running Op 3; Available at Time 54; Executed 54 to 60; Deadline: None>
  <Event 5; Running Op 4; Available at Time 60; Executed 60 to 68; Deadline: None>
]
Worker2: [
  <Event 2; Running Op 2; Available at Time 14; Executed 14 to 18; Deadline: None>
  <Event 5; Running Op 2; Available at Time 50; Executed 50 to 54; Deadline: None>
]


In [16]:
wp2.reset()
simulate(fifo_schedule,[Event(2,0,4),Event(5,0,4)],wp2,lat,100)
print(wp2.history())

Activate: <Event 2; Running Op 0; Available at Time 4; Executed None to None; Deadline: None>
Activate: <Event 5; Running Op 0; Available at Time 4; Executed None to None; Deadline: None>
finished event: <Event 2; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: None>
finished event: <Event 5; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: None>
finished event: <Event 2; Running Op 1; Available at Time 14; Executed 14 to 16; Deadline: None>
finished event: <Event 5; Running Op 1; Available at Time 14; Executed 16 to 18; Deadline: None>
finished event: <Event 2; Running Op 2; Available at Time 14; Executed 14 to 18; Deadline: None>
finished event: <Event 5; Running Op 2; Available at Time 14; Executed 18 to 22; Deadline: None>
finished event: <Event 2; Running Op 3; Available at Time 18; Executed 18 to 24; Deadline: None>
finished event: <Event 2; Running Op 3; Available at Time 22; Executed 22 to 28; Deadline: None>
finished event: <Event 2; Running Op 4; 

## DEADLINES

In [25]:
d_op4 = Operator(4,[],8, relative_deadline = 10)
d_op3 = Operator(3,[4],6,is_join=True, relative_deadline = 6)
d_op2 = Operator(2,[3],4, relative_deadline=4)
d_op1 = Operator(1,[3],2, relative_deadline=5)
d_op0 = Operator(0,[1,2],10, relative_deadline=10)

d_lat = Lattice([d_op0,d_op1,d_op2,d_op3,d_op4])

In [26]:
wp1.reset()
simulate(fifo_schedule,[Event(1,0,4, deadline = 14)], wp1, d_lat , 100)
print(wp1.history())

Activate: <Event 1; Running Op 0; Available at Time 4; Executed None to None; Deadline: 14>
finished event: <Event 1; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: 14>
finished event: <Event 1; Running Op 1; Available at Time 14; Executed 14 to 16; Deadline: 19>
finished event: <Event 1; Running Op 2; Available at Time 14; Executed 16 to 20; Deadline: 18>
finished event: <Event 1; Running Op 3; Available at Time 20; Executed 20 to 26; Deadline: 26>
finished event: <Event 1; Running Op 4; Available at Time 26; Executed 26 to 34; Deadline: 36>
Worker Pool History: 
Worker1: [
  <Event 1; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: 14>
  <Event 1; Running Op 1; Available at Time 14; Executed 14 to 16; Deadline: 19>
  <Event 1; Running Op 2; Available at Time 14; Executed 16 to 20; Deadline: 18>
  <Event 1; Running Op 3; Available at Time 20; Executed 20 to 26; Deadline: 26>
  <Event 1; Running Op 4; Available at Time 26; Executed 26 to 34; Deadline: 36>

In [28]:
wp.reset()
simulate(edf_schedule,[Event(1,0,4, deadline = 14)], wp, d_lat , 100)
print(wp.history())

Activate: <Event 1; Running Op 0; Available at Time 4; Executed None to None; Deadline: 14>
finished event: <Event 1; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: 14>
finished event: <Event 1; Running Op 2; Available at Time 14; Executed 14 to 18; Deadline: 18>
finished event: <Event 1; Running Op 1; Available at Time 14; Executed 18 to 20; Deadline: 19>
finished event: <Event 1; Running Op 3; Available at Time 20; Executed 20 to 26; Deadline: 26>
finished event: <Event 1; Running Op 4; Available at Time 26; Executed 26 to 34; Deadline: 36>
Worker Pool History: 
Worker1: [
  <Event 1; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: 14>
  <Event 1; Running Op 2; Available at Time 14; Executed 14 to 18; Deadline: 18>
  <Event 1; Running Op 1; Available at Time 14; Executed 18 to 20; Deadline: 19>
  <Event 1; Running Op 3; Available at Time 20; Executed 20 to 26; Deadline: 26>
  <Event 1; Running Op 4; Available at Time 26; Executed 26 to 34; Deadline: 36>

In [31]:
wp2.reset()
simulate(edf_schedule,[Event(1,0,4,14)], wp2, d_lat , 300)
print(wp2.history())

Activate: <Event 1; Running Op 0; Available at Time 4; Executed None to None; Deadline: 14>
finished event: <Event 1; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: 14>
finished event: <Event 1; Running Op 1; Available at Time 14; Executed 14 to 16; Deadline: 19>
finished event: <Event 1; Running Op 2; Available at Time 14; Executed 14 to 18; Deadline: 18>
finished event: <Event 1; Running Op 3; Available at Time 18; Executed 18 to 24; Deadline: 24>
finished event: <Event 1; Running Op 4; Available at Time 24; Executed 24 to 32; Deadline: 34>
Worker Pool History: 
Worker1: [
  <Event 1; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: 14>
  <Event 1; Running Op 2; Available at Time 14; Executed 14 to 18; Deadline: 18>
  <Event 1; Running Op 3; Available at Time 18; Executed 18 to 24; Deadline: 24>
  <Event 1; Running Op 4; Available at Time 24; Executed 24 to 32; Deadline: 34>
]
Worker2: [
  <Event 1; Running Op 1; Available at Time 14; Executed 14 to 16; 